# Dataset Creation
1) Download the CNN/Dailymail dataset into the folder `datasets/`. The folder should be named `cnn_dailymail` already, and the train `.csv` should be in `datasets/cnn_dailymail/train.csv` (or change the directory below as needed)
2) create the directory `datasets/cnn_parsed`
3) Run this notebook. This should remove all the dailymail and duplicate articles and save the new train test val split into the folder above.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

dataset_dir_in = 'datasets/cnn_dailymail/'
dataset_dir_out = 'datasets/cnn_parsed/'

In [4]:
# load train dataset
def load_and_parse(dataset = 'train'):
    df = pd.read_csv(dataset_dir_in + dataset + '.csv')
    df = df[df.article.str.contains('CNN')]
    df = df.drop_duplicates('article')
    return df

df_train = load_and_parse('train')
df_test = load_and_parse('test')
df_val = load_and_parse('validation')

In [5]:
df = pd.concat([df_train, df_val])
df_train, df_val = train_test_split(df, test_size=0.3)

In [6]:
df_train.to_csv(dataset_dir_out + 'train.csv')
df_val.to_csv(dataset_dir_out + 'val.csv')
df_test.to_csv(dataset_dir_out + 'test.csv')